In [7]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import json
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [8]:
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests

In [9]:
#PARAMETERS
YEAR_START = 2014
YEAR_END = 2014
TYPE = "feature"

In [10]:
# here we access the webpage and download the content using requests
page=requests.get("http://www.imdb.com/search/title?at=0&sort=alpha&title_type="+TYPE+"&year="+str(YEAR_START)+","+str(YEAR_END))

In [11]:
#First, we get the number of results for this particular year
soup = BeautifulSoup(page.text, "html.parser")
NumResults = soup.find("div", attrs={"class": "leftright"}).find_all("div", attrs={"id": "left"})[0].get_text()
#Output at this point is "\n1-50 of 8,476\ntitles.\n"
NumResults = NumResults.split()[2]

NumResults = int("".join(NumResults.split(',')))   #converting a string of "8,476" to an int of 8476
print NumResults

8997


In [12]:
titles = []
for i in xrange(1,50,50):
    page=requests.get("http://www.imdb.com/search/title?at=0&sort=alpha&start="+str(i)+"&title_type="+TYPE+"&year="+str(YEAR_START)+","+str(YEAR_END))
    soup = BeautifulSoup(page.text, "html.parser")
    rows = soup.find("table", attrs={"class": "results"}).find_all("tr")[1:]
    cleaner = lambda r: [int(r[0].get_text()), r[1].get_text(), r[2].get_text(), r[2].find("a").get("href")]
    titles = titles + [(row.find("td", attrs={"class":"title"}).find("a").get("href")) for row in rows]

print len(titles)

50


In [13]:
urlcache={} #a dict to store the visited urls

In [14]:
def get_page(url):
    # Check if URL has already been visited.
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            r = requests.get("http://www.imdb.com/%s" % url)

            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]

In [15]:
start = time.time()
for title in titles:
    get_page(title)
print time.time() - start

81.6770000458


In [16]:
print np.sum([(urlcache[k]==1) or (urlcache[k]==2) for k in urlcache])# no one or two's
print len(titles)==len(urlcache)#we got all of the urls

0
True


In [17]:
def movie_info(url,page_text):
    info = {}
    info['url'] = url  
    soup = BeautifulSoup(page_text, "html.parser")
    
    box = soup.find("div", attrs={"id": "full_subnav"})
    if box:
        rows = box.find_all("li")

        for row in rows:
            #print row
            #print row.find("a").get_text()
            if row.find("a").get_text() == "Filming Locations":
                #print row.find("a", attrs={"class": "link ghost"})
                if row.find("a", attrs={"class": "link ghost"}):
                    info['location_page'] = "None"
                else:
                    loc_url = url+'locations?ref_=tt_ql_dt_6'
                    try:
                        r = requests.get("http://www.imdb.com/%s" % loc_url)
                        time.sleep(1)
                        if r.status_code == 200:
                            #print 'here'
                            soup2 = BeautifulSoup(r.text,"html.parser")
                            locations = soup2.find_all("div", attrs={"class": "soda"})
                            #print locations
                            temp = []
                            for location in locations:
                                #print location
                                temp.append(location.find("a").get_text().replace("\n", " "))
                            #print temp
                            info['location_page'] = temp   
                        else:
                            info['location_page'] = "fail"
                    except:
                        info['location_page'] = "fail"

    topbar = soup.find("table", attrs={"id": "title-overview-widget-layout"})
    
    #get information from the first info box
    if topbar:
        infobar = topbar.find("div", attrs={"class": "infobar"})
        if infobar:
            if infobar.find("meta", attrs={"itemprop": "contentRating"}):
                info['contentRating'] = infobar.find("meta", attrs={"itemprop": "contentRating"}).get_text().split()[0]
            if infobar.find("time", attrs={"itemprop": "duration"}):
                info['duration'] = infobar.find("time", attrs={"itemprop": "duration"}).get_text().strip()
            genres = infobar.find_all("span", attrs={"itemprop": "genre"})
            temp = []
            for genre in genres:
                temp.append(genre.get_text())
            info['genre'] = temp
            
            #print infobar.find("meta", attrs={"itemprop": "datePublished"}).
            temp = []            
            release_dates = infobar.find_all("a", attrs={"title": "See all release dates"})
            for dates in release_dates:
                temp.append(dates.get_text().replace("\n", " "))                
            info['release_dates'] = temp

    #get name of movie
    if topbar.find("td", attrs={"id": "overview-top"}).find("h1").find("span", attrs={"itemprop":"name"}):
        info['name'] = topbar.find("td", attrs={"id": "overview-top"}).find("h1").find("span", attrs={"itemprop":"name"}).get_text()
        info['year'] = topbar.find("td", attrs={"id": "overview-top"}).find("h1").find("span", attrs={"class":"nobr"}).get_text()[1:-1]

        
    starbox = topbar.find("div", attrs={"class": "star-box-details"})
    if starbox:
        if starbox.find("span", attrs={"itemprop": "ratingValue"}):
            info['user_ratings'] = starbox.find("span", attrs={"itemprop": "ratingValue"}).get_text()
        if starbox.find("span", attrs={"itemprop": "ratingCount"}):
            info['user_ratings_count'] = starbox.find("span", attrs={"itemprop": "ratingCount"}).get_text()        
        if starbox.find("a", attrs={"href": "criticreviews?ref_=tt_ov_rt"}):
            info['critic_ratings'] = starbox.find("a", attrs={"href": "criticreviews?ref_=tt_ov_rt"}).get_text().strip()      
            
    detailsbox = soup.find("div", attrs={"id": "titleDetails"})
    if detailsbox:
        txtblocks = detailsbox.find_all("div", attrs={"class": "txt-block"})
        for block in txtblocks:
            #print block
            if block.find("h4"):
                if block.find("h4").get_text() == "Country:":
                    info['country'] = block.find("a").get_text()
                if block.find("h4").get_text() == "Language:":
                    info['language'] = block.find("a").get_text()
                if block.find("h4").get_text() == "Budget:":
                    info['budget'] = block.get_text().split()[1]
                if block.find("h4").get_text() == "Opening Weekend:":
                    info['opening_weekend'] = block.get_text().split()[2]
                if block.find("h4").get_text() == "Gross:":
                    info['gross'] = block.get_text().split()[1]
                    #print info['gross']
               
    
    #print info
    return info     

#Testing Code
#k = '/title/tt1951264/'
#r = requests.get("http://www.imdb.com"+k)
#v = r.text

#movie_info(k, v)

In [18]:
movie_info_list=[]

for k,v in urlcache.items():
    movie_info_list.append(movie_info(k, v))

In [19]:
movie_info_list

[{'country': u'Kuwait',
  'duration': u'75 min',
  'genre': [u'Drama', u'Thriller'],
  'language': u'Arabic',
  'location_page': 'None',
  'name': u'090',
  'release_dates': [u' 31 December 2014 (Kuwait) '],
  'url': u'/title/tt4305760/',
  'user_ratings': u'3.9',
  'user_ratings_count': u'36',
  'year': u'2014'},
 {'country': u'Italy',
  'genre': [u'Sci-Fi'],
  'language': u'Italian',
  'location_page': 'None',
  'name': u'12 12 12',
  'release_dates': [u' 6 March 2014 (Italy) '],
  'url': u'/title/tt2353755/',
  'user_ratings': u'2.3',
  'user_ratings_count': u'19',
  'year': u'2014'},
 {'country': u'USA',
  'genre': [u'Comedy', u'Romance'],
  'language': u'English',
  'location_page': [u'Los Angeles, California, USA '],
  'name': u'10 Things I Hate About Life',
  'release_dates': [u' 2014 (USA) '],
  'url': u'/title/tt2402917/',
  'year': u'2014'},
 {'contentRating': u'R',
  'country': u'Spain',
  'critic_ratings': u'75/100',
  'duration': u'99 min',
  'genre': [u'Comedy', u'Drama',

In [20]:
fd = open("tempdata/movieinfo.json","w")
json.dump(movie_info_list, fd)
fd.close()

In [21]:
df = pd.DataFrame(movie_info_list)

In [22]:
df

,budget,contentRating,country,critic_ratings,duration,genre,gross,language,location_page,name,opening_weekend,release_dates,url,user_ratings,user_ratings_count,year
0,NaN,NaN,Kuwait,NaN,75 min,"[Drama, Thriller]",NaN,Arabic,None,090,NaN,[ 31 December 2014 (Kuwait) ],/title/tt4305760/,3.9,36,2014
1,NaN,NaN,Italy,NaN,NaN,[Sci-Fi],NaN,Italian,None,12 12 12,NaN,[ 6 March 2014 (Italy) ],/title/tt2353755/,2.3,19,2014
2,NaN,NaN,USA,NaN,NaN,"[Comedy, Romance]",NaN,English,"[Los Angeles, California, USA ]",10 Things I Hate About Life,NaN,[ 2014 (USA) ],/title/tt2402917/,NaN,NaN,2014
3,NaN,R,Spain,75/100,99 min,"[Comedy, Drama, Romance]",NaN,Spanish,"[Barcelona, Catalonia, Spain , Barcelona, Cata...",10.000 Km,NaN,[ 10 July 2015 (USA) ],/title/tt3114132/,6.8,"2,136",2014
4,NaN,R,UK,83/100,99 min,"[Action, Drama, Thriller]","$1,268,760",English,"[Park Hill Estate, Sheffield, South Yorkshire,...",'71,"$55,761",[ 10 October 2014 (UK) ],/title/tt2614684/,7.2,"28,183",2014
5,NaN,NaN,USA,NaN,78 min,"[Comedy, Romance]",NaN,English,None,'Til Love Comes,NaN,[],/title/tt4199762/,NaN,NaN,2014
6,NaN,NaN,Taiwan,NaN,110 min,"[Drama, Romance]",NaN,Mandarin,"[Tai-tung, Taiwan , Hua-lian, Taiwan , Taipei,...",Han chan xiao ying,NaN,[ 10 October 2014 (China) ],/title/tt4024410/,6.0,51,2014
7,NaN,NaN,Turkey,NaN,105 min,[Comedy],NaN,Turkish,None,10. Köy Teyatora,NaN,[ 2 May 2014 (Turkey) ],/title/tt3746926/,4.1,56,2014
8,IDR,NaN,Indonesia,NaN,110 min,"[Drama, Family, Music]",NaN,Indonesian,"[Bontang, Indonesia , Jakarta, Indonesia ]",12 Menit,NaN,[ 29 January 2014 (Indonesia) ],/title/tt3168514/,8.2,57,2014
9,NaN,NaN,Philippines,NaN,NaN,[Drama],NaN,Filipino,None,#Y,NaN,[ 10 December 2014 (Philippines) ],/title/tt3887700/,7.0,34,2014
